In [1]:
import sys
sys.executable

'd:\\DA projects\\walmart-sales-analytics-clean\\venv\\Scripts\\python.exe'

In [2]:
import sys
from pathlib import Path

# Add project root to Python path
PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.preprocessing import (
    clean_columns,
    convert_date,
    handle_missing,
    basic_outlier_handling
)

from src.utils import summarize


# Walmart Sales — EDA & Business Storytelling

**Objective:** Perform exploratory data analysis on the Walmart sample dataset to extract business KPIs, understand trends, test promotion/holiday effects, and prepare visuals for a one-page Power BI executive dashboard.

**Deliverables in this notebook:**
- Cleaned sample dataframe for analysis
- Top-level KPIs (total sales, by store, by dept, avg basket)
- Time-series analysis (weekly/monthly seasonality)
- Promotion & holiday impact analysis
- SQL-equivalent queries (for later use in warehouse)
- Short business recommendations


In [3]:
# standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# display settings
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 180)
sns.set(style="whitegrid")

# plotting defaults
plt.rcParams["figure.figsize"] = (10, 5)


In [4]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("../data/sample")
SAMPLE_PATH = DATA_DIR / "walmart_sample.csv"

assert SAMPLE_PATH.exists(), f"Sample file not found: {SAMPLE_PATH}"

df = pd.read_csv(SAMPLE_PATH)

df.shape


(10000, 16)

In [5]:
from src.preprocessing import clean_columns, convert_date, handle_missing

df = clean_columns(df)
df = convert_date(df, date_col="date")
df = handle_missing(df, strategy="simple")

df.head()


D:\DA projects\walmart-sales-analytics-clean\src\preprocessing.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method="ffill").fillna(method="bfill")
D:\DA projects\walmart-sales-analytics-clean\src\preprocessing.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df.fillna(method="ffill").fillna(method="bfill")


,store,date,temperature,fuel_price,markdown1,markdown2,markdown3,markdown4,markdown5,cpi,unemployment,dept,weekly_sales,isholiday,type,size,year,month,day,year_month
0,4,2011-11-11,47.12,3.286,12851.52,16658.47,430.13,5630.01,9213.39,129.816710,5.14,13,70002.92,0,0,205863,2011,11,11,2011-11
1,30,2011-04-01,56.36,3.524,0.00,0.00,0.00,0.00,0.00,214.488691,7.93,42,56.90,0,2,42988,2011,4,1,2011-04
2,32,2012-05-25,59.74,3.804,7581.03,0.00,216.17,748.97,3014.25,197.588605,8.09,11,13227.53,0,0,203007,2012,5,25,2012-05
3,3,2011-12-02,54.97,3.172,850.86,18.94,781.71,394.29,7297.10,222.158952,7.20,26,4273.76,0,1,37392,2011,12,2,2011-12
4,23,2010-11-26,34.95,3.070,0.00,0.00,0.00,0.00,0.00,132.836933,5.29,14,67443.07,1,1,114533,2010,11,26,2010-11


In [6]:
df_raw = pd.read_csv(SAMPLE_PATH)
df_raw.columns

Index(['store', 'date', 'temperature', 'fuel_price', 'markdown1', 'markdown2', 'markdown3', 'markdown4', 'markdown5', 'cpi', 'unemployment', 'dept', 'weekly_sales', 'IsHoliday',
       'type', 'size'],
      dtype='object')

## 📘 Data Dictionary

Below is a brief description of the key columns in the Walmart Sales dataset.

| Column          | Description |
|-----------------|-------------|
| **store**        | Store ID where the sales occurred |
| **date**         | Week-ending date of the sales record |
| **temperature**  | Average temperature in the region for that week |
| **fuel_price**   | Fuel cost for that week |
| **markdown1-5**  | Promotional markdowns / discounts applied to products |
| **cpi**          | Consumer Price Index (inflation indicator) |
| **unemployment** | Regional unemployment rate |
| **dept**         | Product department ID |
| **weekly_sales** | Total weekly sales for that store & department |
| **is_holiday**   | Whether the week contains a major holiday |
| **type**         | Store type classification (A, B, C) |
| **size**         | Store size in sq-ft |
| **year**         | Extracted from `date` |
| **month**        | Extracted from `date` |
| **week**         | ISO week number |
| **dayofweek**    | Day name (Mon, Tue, etc.) |
| **month_num**    | Extracted numeric month |

This dictionary will help guide our EDA and KPI definitions.

In [7]:
print("Dataset shape:", df.shape)

# Summary of missing values
missing = df.isna().sum().sort_values(ascending=False)
missing

Dataset shape: (10000, 20)


store           0
date            0
temperature     0
fuel_price      0
markdown1       0
markdown2       0
markdown3       0
markdown4       0
markdown5       0
cpi             0
unemployment    0
dept            0
weekly_sales    0
isholiday       0
type            0
size            0
year            0
month           0
day             0
year_month      0
dtype: int64

### 🧹 Missing Values & Data Quality

- The dataset contains **no missing values** across key numerical and categorical columns.
- This indicates a clean and analysis-ready sample.
- No imputation or row removal was required at this stage.


In [8]:
df.dtypes

store                    int64
date            datetime64[ns]
temperature            float64
fuel_price             float64
markdown1              float64
markdown2              float64
markdown3              float64
markdown4              float64
markdown5              float64
cpi                    float64
unemployment           float64
dept                     int64
weekly_sales           float64
isholiday                int64
type                     int64
size                     int64
year                     int32
month                    int32
day                      int32
year_month           period[M]
dtype: object

In [9]:
df.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
store,10000.0,NaN,NaN,NaN,22.1326,1.0,11.0,22.0,33.0,45.0,12.861071
date,10000,NaN,NaN,NaN,2011-06-18 15:27:47.520000256,2010-02-05 00:00:00,2010-10-15 00:00:00,2011-06-17 00:00:00,2012-02-24 00:00:00,2012-10-26 00:00:00,NaN
temperature,10000.0,NaN,NaN,NaN,60.217451,-2.06,46.97,62.185,74.4325,100.14,18.520471
fuel_price,10000.0,NaN,NaN,NaN,3.359872,2.472,2.938,3.445,3.73825,4.468,0.457614
markdown1,10000.0,NaN,NaN,NaN,2558.948811,0.0,0.0,0.0,2809.05,88646.76,6012.447091
markdown2,10000.0,NaN,NaN,NaN,969.985863,0.0,0.0,0.0,1.53,104519.54,5429.824609
markdown3,10000.0,NaN,NaN,NaN,478.814946,0.0,0.0,0.0,4.54,109030.75,5637.167188
markdown4,10000.0,NaN,NaN,NaN,1069.418662,0.0,0.0,0.0,427.39,67474.85,3898.320235
markdown5,10000.0,NaN,NaN,NaN,1661.516529,0.0,0.0,0.0,2153.36,108519.28,4589.826199
cpi,10000.0,NaN,NaN,NaN,171.651746,126.064,132.223032,182.667615,212.836564,227.232807,39.120269


## 📊 Key Performance Indicators (KPIs)
This section summarizes the core business metrics for Walmart weekly sales.


In [10]:
#KPI 1: Total Revenue
total_revenue = df['weekly_sales'].sum()
total_revenue
f"{total_revenue:,.2f}"

'160,338,653.43'

In [11]:
#KPI 2 — Revenue by Store (Top 10)
rev_by_store = (
    df.groupby('store')['weekly_sales']
      .sum()
      .sort_values(ascending=False)
)
rev_by_store.head(10)

store
10    7720674.84
14    7043847.18
13    6895967.09
2     6671728.28
4     6606799.21
20    6435671.06
27    5857620.97
39    5825502.39
1     5573499.66
24    5196957.73
Name: weekly_sales, dtype: float64

In [12]:
#KPI 3 — Revenue by Department
rev_by_dept = (
    df.groupby('dept')['weekly_sales']
      .sum()
      .sort_values(ascending=False)
)
rev_by_dept.head(10)

dept
92    10862507.14
95    10738136.55
38     9230273.38
72     8310613.32
40     6938159.52
2      6840111.52
90     5696264.07
91     5569236.04
13     4779619.48
8      4445656.87
Name: weekly_sales, dtype: float64

In [13]:
#KPI 4 — Average Weekly Sales
avg_weekly_sales = df['weekly_sales'].mean()
avg_weekly_sales
f"{avg_weekly_sales:,.2f}"

'16,033.87'

In [23]:
#KPI 5 — Holiday vs Non-Holiday Sales
holiday_sales = df[df['isholiday'] == True]['weekly_sales'].mean()
non_holiday_sales = df[df['isholiday'] == False]['weekly_sales'].mean()

holiday_sales, non_holiday_sales



(np.float64(17681.355396600567), np.float64(15908.717077684529))

In [24]:
{
    "Holiday Week Avg": holiday_sales,
    "Non-Holiday Week Avg": non_holiday_sales,
    "Lift (%)": (holiday_sales - non_holiday_sales) / non_holiday_sales * 100
}

{'Holiday Week Avg': np.float64(17681.355396600567),
 'Non-Holiday Week Avg': np.float64(15908.717077684529),
 'Lift (%)': np.float64(11.142559832197616)}

In [25]:
#KPI 6 — Store Type Performance (A, B, C)
rev_by_type = df.groupby('type')['weekly_sales'].sum()
rev_by_type

type
0    1.031541e+08
1    4.796921e+07
2    9.215377e+06
Name: weekly_sales, dtype: float64

In [26]:
#KPI 7 — Store Size Impact
df.groupby('store')[['size', 'weekly_sales']].sum().corr()

,size,weekly_sales
size,1.000000,0.855949
weekly_sales,0.855949,1.000000


In [ ]:
#KPI 8 — Sales per Store per Week
sales_per_store_week = (
    df.groupby(['store', 'week'])['weekly_sales']
      .sum()
      .reset_index()
)
sales_per_store_week.head()


### 📝 KPI Insights

- Store X generates the highest revenue, indicating strong demand or larger customer base.
- Department Y contributes the most to overall sales.
- Holiday vs non-holiday comparison shows that:
  - Holiday weeks increase/decrease revenue by Z%.
- Store Type A significantly outperforms B and C.
- Store size shows a moderate/strong/weak correlation with sales.


In [19]:
# Create promotion flag
markdown_cols = [c for c in df.columns if c.startswith("markdown")]
df['has_promotion'] = df[markdown_cols].sum(axis=1) > 0

promo_comparison = df.groupby('has_promotion')['weekly_sales'].mean()
promo_comparison

has_promotion
False    15980.14738
True     16130.23948
Name: weekly_sales, dtype: float64

### 🎯 Promotion Impact Analysis

- Weeks with promotions show (higher/lower) average sales.
- This suggests promotions have a measurable impact on demand.
- Further A/B testing is recommended to isolate causality.


### 🔁 SQL-Equivalent Analysis (Conceptual)

The following pandas operations directly map to SQL queries using:
- `GROUP BY`
- `SUM`, `AVG`
- Date-based aggregations

These transformations will later be implemented in SQL.


In [20]:
df.groupby('store')['weekly_sales'].sum()

store
1     5573499.66
2     6671728.28
3     1199449.52
4     6606799.21
5     1066994.32
6     4974501.89
7     1839258.31
8     3224407.22
9     2103719.57
10    7720674.84
11    4839644.32
12    3266940.72
13    6895967.09
14    7043847.18
15    2282508.58
16    1731623.44
17    2753331.10
18    3968783.91
19    4936737.90
20    6435671.06
21    2663652.02
22    3412154.57
23    4162244.74
24    5196957.73
25    2441823.19
26    3822585.88
27    5857620.97
28    5113703.20
29    1914653.14
30    1403282.73
31    4760094.17
32    3473941.23
33     807834.19
34    2375739.18
35    2312443.35
36    1434884.98
37    1845140.43
38    1069398.17
39    5825502.39
40    3285449.07
41    3996944.70
42    1433396.53
43    2359051.98
44    1105107.30
45    3128959.47
Name: weekly_sales, dtype: float64

In [21]:
# SQL equivalent:
# SELECT store, SUM(weekly_sales) FROM sales GROUP BY store;

## 📌 Final Business Insights

- A small number of stores contribute disproportionately to total revenue.
- Certain departments consistently outperform others.
- Sales exhibit strong seasonal patterns across months and weeks.
- Promotional weeks generally outperform non-promotional weeks.
- Store type and size influence overall sales performance.

These insights form the basis for dashboard KPIs and future forecasting models.


In [22]:
import json
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
DOCS_DIR = PROJECT_ROOT / "docs"
DOCS_DIR.mkdir(exist_ok=True)

monthly_trend = (
    df.groupby(df["date"].dt.to_period("M"))["weekly_sales"]
      .sum()
      .rename_axis("month")
      .reset_index()
)

monthly_trend["month"] = monthly_trend["month"].astype(str)

kpis = {
    "total_sales": float(df["weekly_sales"].sum()),
    "avg_weekly_sales": float(df["weekly_sales"].mean()),
    "best_store": int(df.groupby("store")["weekly_sales"].sum().idxmax()),
    "best_store_sales": float(df.groupby("store")["weekly_sales"].sum().max()),
    "worst_store": int(df.groupby("store")["weekly_sales"].sum().idxmin()),
    "monthly_trend": dict(
        zip(monthly_trend["month"], monthly_trend["weekly_sales"])
    )
}

with open(DOCS_DIR/"metrics.json", "w") as f:
    json.dump(kpis, f, indent=4)

kpis


{'total_sales': 160338653.43,
 'avg_weekly_sales': 16033.865343000001,
 'best_store': 10,
 'best_store_sales': 7720674.84,
 'worst_store': 33,
 'monthly_trend': {'2010-02': 4878672.77,
  '2010-03': 3681281.17,
  '2010-04': 5954646.87,
  '2010-05': 4381718.31,
  '2010-06': 4641671.45,
  '2010-07': 5278187.23,
  '2010-08': 4250038.3,
  '2010-09': 3846633.93,
  '2010-10': 4624714.06,
  '2010-11': 6167769.58,
  '2010-12': 7748681.4,
  '2011-01': 3969686.52,
  '2011-02': 4656248.18,
  '2011-03': 4124833.06,
  '2011-04': 5416902.56,
  '2011-05': 4275968.35,
  '2011-06': 4256520.52,
  '2011-07': 5265796.4,
  '2011-08': 4841959.69,
  '2011-09': 5633652.0,
  '2011-10': 3544861.43,
  '2011-11': 5131612.82,
  '2011-12': 5946623.49,
  '2012-01': 4262661.64,
  '2012-02': 3673746.08,
  '2012-03': 5460046.77,
  '2012-04': 4728774.92,
  '2012-05': 4123958.45,
  '2012-06': 6556283.24,
  '2012-07': 3930392.03,
  '2012-08': 6242969.28,
  '2012-09': 4100635.54,
  '2012-10': 4740505.39}}